In [1]:
import jsonlines
import tqdm
import pickle

In [3]:
artist_reviews = {}

In [4]:
with jsonlines.open('../data_for_558_proj/songkick.jl') as reader:
    for obj in reader:
        if len(obj["reviews"]) > 0:
            text = " ".join(obj["reviews"])
            text = " ".join(text.split()).strip()
            artist_reviews[obj["url"]] = {"text" : text,
                                          "count" : len(obj["reviews"]),
                                          "name" : obj["name"]}

In [11]:
# artist_reviews["https://www.songkick.com/artists/268425-beach-boys"]["text"]

In [9]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

In [13]:
src_text = [artist_reviews[key]["text"] for key in artist_reviews]

In [19]:
batch_size = 32
model_name = 'google/pegasus-xsum'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

In [20]:
model.to(torch_device)

PegasusForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): SinusoidalPositionalEmbedding(512, 1024)
      (layers): ModuleList(
        (0): EncoderLayer(
          (self_attn): Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )


In [22]:
# torch_device

In [24]:
summary_text = []
for i in tqdm.tqdm(range(0, len(src_text), batch_size)):
    batch = src_text[i:i+batch_size]
    batch = tokenizer.prepare_seq2seq_batch(batch, truncation=True, padding='longest').to(torch_device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    summary = summary + tgt_text

  0%|          | 0/156 [00:02<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 10.92 GiB total capacity; 9.64 GiB already allocated; 470.50 MiB free; 9.65 GiB reserved in total by PyTorch)

['ab', 'bc', 'cd']